In [ ]:
%%requirejs /static/requirejs@2.3.1/require.min.js
$           jquery  /static/jquery@2.1.1/jquery.min.js
React       react   /static/react@15.3.1/build/react.js
ReactDOM            /static/react@15.3.1/build/react-dom.min.js
R                   /static/ramda@0.22.1/ramda.min.js
d3                  /static/d3@4.2.2/d3.min.js
snap                /static/snapsvg@0.4.1/dist/snap.svg-min.js
vis                 /static/visjs@4.16.1/dist/vis.min.js

In [ ]:
import nbformat
import extensions.my_ext

# import importlib
# importlib.reload(extensions.my_ext)

extensions.my_ext.load_ipython_extension(get_ipython())

def make(fname):
    source = open(fname, 'rb').read()
    nb = nbformat.reads(source.decode('utf8'), as_version=4)
    parents = {}
    cells = [extensions.my_ext.NotebookCell(cell, parents, index) for index, cell in enumerate(nb.cells)]
    with open('/data/sets/static/p/gchatstats/upload.html', 'w') as f:
        f.write(make_html(cells))

def make_html(cells):
    header_html = ''
    body_html = ''
    jsx_scripts = ''
    scripts = ''
    run_scripts = ''
    for cell in cells:
        if cell.magic == '%%js' and '@out' in cell.labels:
            scripts += cell.get_output('application/javascript') + '\n'
        if cell.magic == '%%js' and '@out_run' in cell.labels:
            run_scripts += cell.get_output('application/javascript') + '\n'
        elif cell.magic == '%%jsx' and '@out' in cell.labels:
            jsx_scripts += cell.get_output('application/javascript') + '\n'
        elif cell.magic == '%%css' and '@out' in cell.labels:
            header_html += cell.get_output('text/html') + '\n'
        elif cell.magic == '%%html' and '@header' in cell.labels:
            header_html += cell.source + '\n'
        elif cell.magic == '%%html' and '@body' in cell.labels:
            body_html += cell.source + '\n'
        elif cell.kind == 'code' and cell.source.startswith('%css '):
            header_html += cell.get_output('text/html') + '\n'
        elif cell.magic == '%%requirejs':
            require_config = cell.get_output('application/javascript') + '\n'
    res = '<!DOCTYPE html><html><head>\n'
    res += header_html + '\n'
    if hasattr(extensions.my_ext, 'requirejs'):
        res += '<script src=\"%s\"></script>\n' % extensions.my_ext.requirejs['lib_path']
    res += '<script>\n' + require_config + '\n</script>\n'
    res += '<script>\n' + scripts + '\n</script>\n'
    res += '<script>\n' + jsx_scripts + '\n</script>\n'
    res += '</head><body>\n'
    res += body_html
    res += '<script>\n' + run_scripts + '\n</script>\n'
    res += '</body></html>\n'
    return res

In [31]:
%%file /data/sets/venv/jupyter/share/jupyter/nbextensions/jupyter_extension/jupyter_extension.yaml
Type: IPython Notebook Extension
Name: Custom jupyter extension
Main: main.js
Compatibility: 4.x

Writing /data/sets/venv/jupyter/share/jupyter/nbextensions/jupyter_extension/jupyter_extension.yaml


In [76]:
%%file /data/sets/venv/jupyter/share/jupyter/nbextensions/jupyter_extension/main.js
define(['base/js/namespace'], function(Jupyter){
    function modify_magic() {
        var modes = Jupyter.CodeCell.options_default.highlight_modes;
        modes.magic_javascript = {reg: ["^%%javascript", "^%%js", "^%%jsx"]};
        modes.magic_css = {reg: ["^%%css"]};
        modes.magic_python = {reg: ['^%%python3?', '^%%py']};
        modes.magic_html = {reg: ['^%%html']};
    }
    function register_output_raw_cells() {
        var mime = ['text/html', 'application/javascript'];
        var handler = function () {
            var cur_cell = Jupyter.notebook.get_selected_cell();
            if (!('output_area' in cur_cell)) {
                return;
            }
            var cell_outputs = cur_cell.output_area.outputs;
            for (var i=0; i<cell_outputs.length; i++) {
                var data = cell_outputs[i].data;
                if (typeof data != 'object') continue;
                for (var j=0; j<mime.length; j++) {
                    if (mime[j] in data) {
                        Jupyter.pager.clear();
                        Jupyter.pager.expand();
                        Jupyter.pager.append_text(mime[j]);
                        Jupyter.pager.append_text(data[mime[j]]);
                        break;
                    }
                }
                if (j == mime.length) {
                    var res = []
                    for (var m in data) {
                        res.push(m);
                    }
                    console.log(res);
                }
            }
        };
        var action = {
            icon: 'fa-code',
            help    : 'inspect output source',
            help_index : 'zz',
            handler : handler
        };
        var prefix = 'auto';
        var action_name = 'inspect-output-source';
        var full_action_name = Jupyter.actions.register(action, action_name, prefix);
        Jupyter.keyboard_manager.command_shortcuts.add_shortcut('Alt-O', full_action_name);        
    }
    function bind_checkpoint() {
        if (!window.checkpoint_bound) {
            window.checkpoint_bound = true;
            Jupyter.notebook.events.on('checkpoint_created.Notebook', () => {on_checkpoint()});
            console.log('binding checkpoint');
        }
        window.on_checkpoint = function () {
            IPython.notebook.kernel.execute(
               'if "make" in globals():\n\tmake("' + IPython.notebook.notebook_path + '")',
               {iopub: {output: function(obj) {
                   if (obj.msg_type == 'error') {
                       console.log(obj);
                       alert(obj.content.ename + ': ' + obj.content.evalue);
                   }
               }}}
            );
        }
    }
    function bind_refresh_highlight() {
        /*
        if (!window.refresh_highlight_bound) {
            window.refresh_highlight_bound = true;
            Jupyter.notebook.events.on('notebook_loaded.Notebook', () => {refresh_highlight()});
            console.log('binding checkpoint');
        }
        */
        window.refresh_highlight = function () {
            console.log("refreshing highlight")
            var cells = IPython.notebook.get_cells();
            for (var i=0; i<cells.length; i++) {
                cells[i].force_highlight();
            }
        }
        refresh_highlight();
    }
    function load_ipython_extension() {
        modify_magic();
        register_output_raw_cells();
        bind_checkpoint();
        bind_refresh_highlight();
    }
    return {
        load_ipython_extension: load_ipython_extension
    };
});

Overwriting /data/sets/venv/jupyter/share/jupyter/nbextensions/jupyter_extension/main.js


In [69]:
%load_ext my_ext

The my_ext extension is already loaded. To reload it, use:
  %reload_ext my_ext


In [86]:
%%file ~/.ipython/extensions/my_ext.py

import re
import nbformat
import json
import subprocess
from IPython.core.magic import register_cell_magic, register_line_magic
from IPython.core.display import display, Javascript, HTML

def load_ipython_extension(IPython):

    @register_cell_magic('py')
    def css_cell_magic(line, cell):
        IPython.kernel.shell.run_cell(cell, False)

    completer = IPython.Completer.splitter
    completer.delims = completer.delims.replace('@', '')

    @register_cell_magic('requirejs')
    def requirejs_cell_magic(line, cell):
        global requirejs
        requirejs = {
            'config': {},
            'shortcuts': {},
            'lib_path' : line.split()[0]
        }
        for line in cell.split('\n'):
            elems = line.split()
            if len(elems) == 2:
                shortcut, file = elems
                requirejs['shortcuts'][shortcut] = file
            elif len(elems) == 3:
                shortcut, import_name, file = elems
                requirejs['shortcuts'][shortcut] = import_name
                if file.endswith('.js'):
                    file = file[:-3]
                requirejs['config'][import_name] = file
        if len(requirejs['config']) > 0:
            paths = json.dumps(requirejs['config'], indent=4)
            return Javascript('require.config({paths:\n %s\n});' % paths)

    def add_requirejs(cell):
        if 'requirejs' not in globals():
            return cell
        shortcuts, files = zip(*requirejs['shortcuts'].items())
        files = ', '.join(['"%s"' % file for file in files])
        shortcuts = ', '.join([shortcut for shortcut in shortcuts])
        return 'require([%s], function(%s) {\n%s\n})' % (files, shortcuts, cell)

    def js_no_out(code):
        code += '; IPython.notebook.events.one("output_appended.OutputArea", function() {'
        code += '     setTimeout(function() {'
        code += '         console.log("output_appended");'
        code += '         var idx = IPython.notebook.get_selected_index() - 1;'
        code += '         IPython.notebook.get_cell(idx).clear_output();'
        code += '     }, 1000)'
        code += '})'
        return code
    
    @register_cell_magic('js')
    def css_cell_magic(line, cell):
        res = add_requirejs(cell)
        if line.startswith('no_out'):
            res = js_no_out(res)
        return Javascript(res)

    def prefix_css_file(fname, prefix):
        res = subprocess.run(['/data/sets/venv/node_modules/.bin/prefix-css', prefix, fname], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        if res.stderr:
            raise Exception(res.stderr.decode('utf8'))
        return res.stdout.decode('utf8')

    def prefix_css_source(source, prefix):
        temp_fname = '/data/temp/prefix.css'
        with open(temp_fname, 'w') as f:
            f.write(source)
        return prefix_css_file(temp_fname, prefix)

    def split_by_token(line):
        line = line.strip()
        if line == '':
            return ('', '')
        elif line[0] == '"':
            pos = line.index('"', 1)
            return (line[1:pos], line[pos+1:])
        elif ' ' in line:
            return line.split(' ', 1)
        else:
            return (line, '')

    @register_cell_magic('css')
    def css_cell_magic(line, cell):
        prefix, _ = split_by_token(line)
        if prefix != '' and not prefix.startswith('@'):
            source = prefix_css_source(cell, prefix)
        else:
            source = cell
        try:
            return HTML('<style>\n' + source + '\n</style>')
        except Exception as e:
            return HTML('<pre>\n' + e.args[0] + '\n</pre>')

    @register_line_magic('css')
    def css_line_magic(line):
        fname, rest = split_by_token(line)
        prefix, _ = split_by_token(rest)
        source = open(fname).read()
        if prefix != '' and not prefix.startswith('@'):
            source = prefix_css_source(source, prefix)
        try:
            return HTML('<style>\n' + source + '\n</style>')
        except Exception as e:
            return HTML('<pre>\n' + e.args[0] + '\n</pre>')

    def register_react_worker():
        node_source_location = '/data/temp/node_react_worker.js'
        node_modules_location = '/data/sets/venv/node_modules'
        Bg.inst('react', force_new=True).set_params(
            ['node', node_source_location],
            stdin=subprocess.PIPE,
            stderr=subprocess.PIPE,
            stdout=subprocess.PIPE,
            env = {'NODE_PATH': node_modules_location}
        )

    def react_transform(source):
        inst = Bg.inst('react')
        inst.ensure_running()
        msg = json.dumps(source, ensure_ascii=False).encode('utf8') + b'\n'
        inst.popen.stdin.write(msg)
        inst.popen.stdin.flush()
        res = json.loads(inst.popen.stdout.readline().decode('utf8'))
        if 'result' in res:
            return res['result']
        else:
            raise Exception('Babel error:\n%s' % res['error'])

    register_react_worker()

    @register_cell_magic('jsx')
    def react_magic(line, cell):
        try:
            res = react_transform(add_requirejs(cell))
            if line.startswith('no_out'):
                res = js_no_out(res)
            return Javascript(res)
        except Exception as e:
            return HTML('<pre>' + e.args[0] + '</pre>')

class Bg:
    
    prefix = 'bgprocess_'
    
    def inst(name, force_new=False):
        full_name = Bg.prefix + name
        if full_name in globals():
            old_inst = globals()[full_name]
            if not force_new:
                return old_inst
            else:
                old_inst.stop()
                new_inst = Bg(name).set_params(*old_inst.args, **old_inst.kwargs)
                globals()[full_name] = new_inst
                return new_inst
        else:
            new_inst = Bg(name)
            globals()[full_name] = new_inst
            return new_inst
            
    def __init__(self, name):
        self.name = name
        self.popen = None
        self.err_shown = False
        self.args = ()
        self.kwargs = {}
    
    def is_running(self):
        if self.popen is None:
            return False
        ret_code = self.popen.poll()
        if ret_code is None:
            return True
        return False
    
    def errors(self):
        if self.popen is None:
            return None
        ret_code = self.popen.poll()
        if ret_code is None:
            return None
        if self.err_shown:
            return None
        self.err_shown = True
        err = self.popen.communicate()[1]
        if err is not None:
            return "%s exception (error code %d):\n%s" % (self.name, ret_code, err)
        if ret_code < 0:
            return "%s aborted by signal %d" % (self.name, ret_code)
    
    def stop(self):
        if self.is_running():
            self.popen.kill()
    
    def set_params(self, *args, **kwargs):
        self.args = args
        self.kwargs = kwargs
        return self
    
    def run(self, *args, **kwargs):
        if self.is_running():
            return
        self.popen = subprocess.Popen(*self.args, **self.kwargs)
        self.err_shown = False
    
    def ensure_running(self):
        err = self.errors()
        if err is not None:
            raise Exception(err)
        if not self.is_running():
            print('Starting %s...' % self.name)
            self.run()

class NotebookCell:
    
    def __init__(self, cell, parents, index):
        self.index = index
        self.cell = cell
        self.labels = set()
        self.magic = None
        self.kind = cell['cell_type']
        if self.kind == 'markdown' and '\n' not in cell['source'] and cell['source'].startswith('#'):
            self.kind = 'title'
            self.level = cell['source'].count('#')
            items = list(parents.items())
            for level, _ in items:
                if level >= self.level:
                    del parents[level]
            parents[self.level] = self
            self.labels = set(re.findall('@(\w+)', cell['source']))
            self.source = cell['source'].replace('#', '').split('@')[0].strip()
        self.parents = [parents[level] for level in sorted(parents.keys())]
        if self.kind not in ['code', 'raw']:
            return
        if cell['source'].startswith('%%') and '\n' in cell['source']:
            self.top_line, self.source = cell['source'].split('\n', 1)
            self.magic = self.top_line.split()[0]
            self.labels = set(re.findall('@\w+', self.top_line))
        else:
            self.source = cell['source']

    def last_label(self, *args):
        res = args[0]
        for parent in self.parents:
            inter = parent.labels.intersect(labels)
            if len(inter) > 1:
                raise Exception('More than one label: %s', str(inter))
            elif len(inter) == 1:
                res = inter.pop()
        return res
    
    def index_str(self):
        return '#%d [%s]' % (self.index, str(self.cell['execution_count']))
    
    def snippet(self):
        return self.cell['source'].replace('\n', '; ')[:50]
    
    def get_output(self, mime):
        try:
            for output in self.cell['outputs']:
                if 'data' in output:
                    if mime in output['data']:
                        return output['data'][mime]
            raise Exception()
        except:
            raise Exception('Can\'t get output of cell %s:\n%s' % (self.index_str(), self.snippet()))


Overwriting /home/yaroslav/.ipython/extensions/my_ext.py


In [40]:
%%file /data/temp/node_react_worker.js

var readline = require('readline');
var babel = require("babel-core");

var rl = readline.createInterface({
  input: process.stdin,
  output: process.stdout,
  terminal: false
});

rl.on('line', function(line) {
    try {
        var res = babel.transform(JSON.parse(line), {
            presets: ['es2015', 'react']
        }).code;
        process.stdout.write(JSON.stringify({result: res}) + '\n');
    } catch (e) {
        process.stdout.write(JSON.stringify({error: e.stack}) + '\n');
    }
})

Writing /data/temp/node_react_worker.js


In [87]:
%%file ~/.jupyter/custom/patch.css
.code_cell.init-cell .inner_cell .CodeMirror-gutters {
/*     border-right: 1px solid lightgray; */
    background: #544;
}
.CodeMirror-gutter-wrapper {
    transform: translate(-4px, 0px);
}
.code_cell .inner_cell .CodeMirror-gutters {
/*     border-right: 1px solid black; */
}
.CodeMirror span.cm-qualifier {
    color: lightgray;
}
.CodeMirror span.cm-comment {
    color: gray;
}
div#pager .ui-resizable-handle {
    background: #272727;
    border-color: #777;
}
div#pager .ui-resizable-handle::after {
    border-top: 1px solid #777;
}
#pager-contents {
    background: #111;
}
div#pager-container pre {
    background: #272727;
    box-shadow: 0px 0px 12px 1px rgba(87, 87, 87, 0.2);
    border: none;
    color: #F8F8F8;
}
#tooltip {
    background: #272727;
    border: 1px solid #777;
    color: #F8F8F8;
    box-shadow: 0px 0px 12px 1px rgba(87, 87, 87, 0.2);
}
#tooltip .tooltiparrow{
    color: #F8F8F8;
}

#tooltip .pretooltiparrow:before {
    background-color : #272727;
    border : 1px solid #777;
}

#tooltip .tooltiptext pre {
    background-color : #272727;
    border : none;
    color: #F8F8F8;
}
.fileinput {
    height: 18px
}

Overwriting /home/yaroslav/.jupyter/custom/patch.css


In [88]:
! cd ~/.jupyter/custom && cat dark_theme.css patch.css > custom.css